<a href="https://colab.research.google.com/github/SridharSola/FER/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def get_set_acc(loader, net):
  if torch.cuda.is_available():
    net.cuda()
  test_acc = []
  total_t=0
  correct_t=0
  net.eval() #Needed!!!!
  for data_t, target_t in (loader):

            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net(data_t)
            #print(outputs_t.shape)
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
            test_acc.append(100 * correct_t/total_t) 
  print(f'test acc: {(100 * correct_t/total_t):.4f}\n')

def adjust_learning_rate(optimizer): 
  for param_group in optimizer.param_groups: 
      param_group["lr"] /= 10.

def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

def training(net, train_set, test_set, criterion, learning_rate, epochs, log, model_save):
  if torch.cuda.is_available():
    net.cuda()
  criterion = criterion
  optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate) 
  logfile = open(log, 'w')
  n_epochs = epochs
  print_every = 5
  test_loss_min = np.Inf
  test_loss = []
  test_acc = []
  train_loss = []
  train_acc = []
  best_test_acc = 0.0
  total_step = len(train_set)
  lrs = []
  lrs.append(learning_rate)
  print('\nTraining starting:\n', file = logfile)
  print('\nTraining starting:\n')
  for epoch in range(1, n_epochs+1):
      running_loss = 0.0
      correct = 0
      total=0
      print(f'Epoch {epoch}\n', file = logfile)
      print(f'Epoch {epoch}\n')
      if epoch == 20 or epoch == 28 or epoch==36:     
          #scheduler.step()
          adjust_learning_rate(optimizer)
          lrs.append(optimizer.param_groups[0]["lr"])
          print(f'Updated lr: {lrs[-1]}\n', file = logfile)
          print(f'Updated lr: {lrs[-1]}\n')
      for batch_idx, (data_, target_) in enumerate(train_set):
          data_, target_ = data_.to(device), target_.to(device)
          optimizer.zero_grad()
          
          outputs = net(data_)
          loss = criterion(outputs, target_)
          loss.backward()
          optimizer.step()
          #out, probs = cls(outputs)
          
          running_loss += loss.item()
          _,pred = torch.max(outputs, dim=1)
          correct += torch.sum(pred==target_).item()
          total += target_.size(0)
          if (batch_idx) % 20 == 0:
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                    .format(epoch, n_epochs, batch_idx, total_step, loss.item()), file = logfile)
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                    .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
      train_acc.append(100 * correct / total)
      train_loss.append(running_loss/total_step)
      print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}', file = logfile)
      print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
      batch_loss = 0
      total_t=0
      correct_t=0
      
      with torch.no_grad():
          net.eval()
          for data_t, target_t in (test_set):
              data_t, target_t = data_t.to(device), target_t.to(device)
              outputs_t = net(data_t)
              #outputs_t, probs_t = cls(outputs_t1)
              loss_t = criterion(outputs_t, target_t)
              batch_loss += loss_t.item()
              _,pred_t = torch.max(outputs_t, dim=1)
              correct_t += torch.sum(pred_t==target_t).item()
              total_t += target_t.size(0)
          test_acc.append(100 * correct_t/total_t)
          test_loss.append(batch_loss/len(test_loader))
          network_learned = batch_loss < test_loss_min
          print(f'test loss: {np.mean(test_loss):.4f}, test acc: {(100 * correct_t/total_t):.4f}\n', file = logfile)
          print(f'test loss: {np.mean(test_loss):.4f}, test acc: {(100 * correct_t/total_t):.4f}\n')

          if best_test_acc < test_acc[-1]:
            #Updating the best test_accuracy obtained
            print("Best Test Accuracy Updated", file = logfile)
            print("Best Test Accuracy Updated")
            best_test_acc = test_acc[-1]
          if network_learned:
              test_loss_min = batch_loss
              torch.save(net.state_dict(), model_save)
              print('Improvement-Detected, save-model to drive', file = logfile)
              print('Improvement-Detected, save-model to drive')


      net.train()

  print("The best test accuracy obtained was: ", best_test_acc)
  #End of training

from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

def get_conf_matrix(net, loader):
  y_pred = []
  y_true = []
  net.eval()
  # iterate over test data
  for inputs, labels in loader:
          output = net(inputs) # Feed Network

          output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
          y_pred.extend(output) # Save Prediction
          
          labels = labels.data.cpu().numpy()
          y_true.extend(labels) # Save Truth

  # constant for classes
  classes = ('Surprise', 'Fear', 'Disgust', 'Happiness', 'Sadness', 'Anger', 'Neutral')

  # Build confusion matrix
  cf_matrix = confusion_matrix(y_true, y_pred)
  df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
                      columns = [i for i in classes])

  df_conf_norm = df_cm / df_cm.sum(axis=1)
  plt.figure(figsize = (12,7))
  sn.heatmap(df_conf_norm, annot=True)
  plt.savefig('output.png')

